## Project Final for ECE 556: AI for Radar and Remote Sensing
Paul Delgado

In [1]:
# Python packages
import csv
import datetime
from itertools import product
import json
import os
import pathlib
import requests
import sys
import zipfile

In [2]:
# Third-party packages
#import caffe
import mne_bids
import numpy as np
import openneuro
import PIL
import scipy
from scipy import optimize
import tensorflow as tf

### I) Constants

In [3]:
# Data settings
results_dirpath_str = os.path.normpath(os.path.join(os.getcwd(), 'results'))
print(results_dirpath_str)

C:\Users\PaulDRP\source\repos\eeg1\results


In [4]:
subjects_list = ['S1', 'S2', 'S3']
rois_list = ['VC']
network_name_str = 'VGG19'

In [5]:
# Images in figure 2
image_type = 'natural'
image_label_list = ['Img0002', 'Img0011', 'Img0045', 'Img0048']
max_iteration = 200

In [6]:
# Data Directory
dat_dirpath_str = os.path.normpath(os.path.join(os.getcwd(), '..', 'DeepImageReconstruction'))
print(dat_dirpath_str)

C:\Users\PaulDRP\source\repos\DeepImageReconstruction


In [7]:
decoded_features_dirpath_str = os.path.normpath(os.path.join(dat_dirpath_str, 'data', 'decodedfeatures'))
print(decoded_features_dirpath_str)

C:\Users\PaulDRP\source\repos\DeepImageReconstruction\data\decodedfeatures


In [8]:
img_mean_fp = os.path.normpath(os.path.join(dat_dirpath_str, 'data', 'ilsvrc_2012_mean.npy'))
print(img_mean_fp)

C:\Users\PaulDRP\source\repos\DeepImageReconstruction\data\ilsvrc_2012_mean.npy


In [9]:
# CNN model
model_fp = os.path.normpath(os.path.join(dat_dirpath_str, 'net', 'VGG_ILSVRC_19_layers', 'VGG_ILSVRC_19_layers.caffemodel'))
prototxt_fp = os.path.normpath(os.path.join(dat_dirpath_str, 'net', 'VGG_ILSVRC_19_layers', 'VGG_ILSVRC_19_layers.prototxt'))
channel_swap = (2, 1, 0)
print(model_fp)
print(prototxt_fp)

C:\Users\PaulDRP\source\repos\DeepImageReconstruction\net\VGG_ILSVRC_19_layers\VGG_ILSVRC_19_layers.caffemodel
C:\Users\PaulDRP\source\repos\DeepImageReconstruction\net\VGG_ILSVRC_19_layers\VGG_ILSVRC_19_layers.prototxt


In [10]:
feat_std_fp = os.path.normpath(os.path.join(dat_dirpath_str, 'data', 'estimated_vgg19_cnn_feat_std.mat'))
print(feat_std_fp)

C:\Users\PaulDRP\source\repos\DeepImageReconstruction\data\estimated_vgg19_cnn_feat_std.mat


In [11]:
dataset = 'ds001506'
subject = '01'
session = 'perceptionNaturalImageTraining01'
run = 1
#task = 'trance'
#suffix = 'inplane'
datatype = 'anat'

In [12]:
bids_root_dirpath_str = os.path.normpath(os.path.join(os.getcwd(), '..', dataset))

### II) Helper Functions

In [13]:
def clip_extreme_value(img, pct=1):
    if pct < 0:
        pct = 0.

    if pct > 100:
        pct = 100.

    img = np.clip(img, np.percentile(img, pct/2.), np.percentile(img, 100-pct/2.))
    return img

In [14]:
def estimate_cnn_feat_std(cnn_feat):
    feat_ndim = cnn_feat.ndim
    feat_size = cnn_feat.shape
    # for the case of fc layers
    if feat_ndim == 1 or (feat_ndim == 2 and feat_size[0] == 1) or (feat_ndim == 3 and feat_size[1] == 1 and feat_size[2] == 1):
        cnn_feat_std = np.std(cnn_feat)
    # for the case of conv layers
    elif feat_ndim == 3 and (feat_size[1] > 1 or feat_size[2] > 1):
        num_of_ch = feat_size[0]
        # std for each channel
        cnn_feat_std = np.zeros(num_of_ch, dtype='float32')
        for j in range(num_of_ch):
            feat_ch = cnn_feat[j, :, :]
            cnn_feat_std[j] = np.std(feat_ch)
        cnn_feat_std = np.mean(cnn_feat_std)  # std averaged across channels
    return cnn_feat_std

In [15]:
def L2_loss(feat, feat0, mask=1.):
    d = feat - feat0
    loss = (d*d*mask).sum()
    grad = 2 * d * mask
    return loss, grad

def L1_loss(feat, feat0, mask=1.):
    d = feat - feat0
    loss = np.abs(d*mask).sum()
    grad = np.sign(d)*mask
    return loss, grad

def inner_loss(feat, feat0, mask=1.):
    loss = -(feat*feat0*mask).sum()
    grad = -feat0*mask
    return loss, grad

def gram(feat, mask=1.):
    feat = (feat * mask).reshape(feat.shape[0], -1)
    feat_gram = np.dot(feat, feat.T)
    return feat_gram

def gram_loss(feat, feat0, mask=1.):
    feat_size = feat.shape[:]
    N = feat_size[0]
    M = feat_size[1] * feat_size[2]
    feat_gram = gram(feat, mask)
    feat0_gram = gram(feat0, mask)
    feat = feat.reshape(N, M)
    loss = ((feat_gram - feat0_gram)**2).sum() / (4*(N**2)*(M**2))
    grad = np.dot((feat_gram - feat0_gram), feat).reshape(feat_size) * mask / ((N**2)*(M**2))
    return loss, grad

In [16]:
def switch_loss_fun(loss_type):
    if loss_type == 'l2':
        return L2_loss
    elif loss_type == 'l1':
        return L1_loss
    elif loss_type == 'inner':
        return inner_loss
    elif loss_type == 'gram':
        return gram_loss
    else:
        raise ValueError('unknown loss function type!')

In [17]:
def load_img(path_to_img):
    max_dim = 512
    img = tf.io.read_file(path_to_img)
    img = tf.image.decode_image(img, channels=3)
    img = tf.image.convert_image_dtype(img, tf.float32)
    shape = tf.cast(tf.shape(img)[:-1], tf.float32)
    long_dim = max(shape)
    scale = max_dim / long_dim
    new_shape = tf.cast(shape * scale, tf.int32)
    img = tf.image.resize(img, new_shape)
    img = img[tf.newaxis, :]
    return img

In [18]:
def img_preprocess(img, img_mean=np.float32([104, 117, 123])):
    '''convert to Caffe's input image layout'''
    return np.float32(np.transpose(img, (2, 0, 1))[::-1]) - np.reshape(img_mean, (3, 1, 1))

In [19]:
def create_feature_masks(features, masks=None, channels=None):
    feature_masks = {}
    for layer in features.keys():
        if (masks is None or masks == {} or masks == [] or (layer not in masks.keys())) and (channels is None or channels == {} or channels == [] or (layer not in channels.keys())):  # use all features and all channels
            feature_masks[layer] = np.ones_like(features[layer])
        elif isinstance(masks, dict) and (layer in masks.keys()) and isinstance(masks[layer], np.ndarray) and masks[layer].ndim == 3 and masks[layer].shape[0] == features[layer].shape[0] and masks[layer].shape[1] == features[layer].shape[1] and masks[layer].shape[2] == features[layer].shape[2]:  # 3D mask
            feature_masks[layer] = masks[layer]
        # 1D feat and 1D mask
        elif isinstance(masks, dict) and (layer in masks.keys()) and isinstance(masks[layer], np.ndarray) and features[layer].ndim == 1 and masks[layer].ndim == 1 and masks[layer].shape[0] == features[layer].shape[0]:
            feature_masks[layer] = masks[layer]
        elif (masks is None or masks == {} or masks == [] or (layer not in masks.keys())) and isinstance(channels, dict) and (layer in channels.keys()) and isinstance(channels[layer], np.ndarray) and channels[layer].size > 0:  # select channels
            mask_2D = np.ones_like(features[layer][0])
            mask_3D = np.tile(mask_2D, [len(channels[layer]), 1, 1])
            feature_masks[layer] = np.zeros_like(features[layer])
            feature_masks[layer][channels[layer], :, :] = mask_3D
        # use 2D mask select features for all channels
        elif isinstance(masks, dict) and (layer in masks.keys()) and isinstance(masks[layer], np.ndarray) and masks[layer].ndim == 2 and (channels is None or channels == {} or channels == [] or (layer not in channels.keys())):
            mask_2D_0 = masks[layer]
            mask_size0 = mask_2D_0.shape
            mask_size = features[layer].shape[1:]
            if mask_size0[0] == mask_size[0] and mask_size0[1] == mask_size[1]:
                mask_2D = mask_2D_0
            else:
                mask_2D = np.ones(mask_size)
                n_dim1 = min(mask_size0[0], mask_size[0])
                n_dim2 = min(mask_size0[1], mask_size[1])
                idx0_dim1 = np.arange(n_dim1) + \
                    round((mask_size0[0] - n_dim1)/2)
                idx0_dim2 = np.arange(n_dim2) + \
                    round((mask_size0[1] - n_dim2)/2)
                idx_dim1 = np.arange(n_dim1) + round((mask_size[0] - n_dim1)/2)
                idx_dim2 = np.arange(n_dim2) + round((mask_size[1] - n_dim2)/2)
                mask_2D[idx_dim1, idx_dim2] = mask_2D_0[idx0_dim1, idx0_dim2]
            feature_masks[layer] = np.tile(
                mask_2D, [features[layer].shape[0], 1, 1])
        else:
            feature_masks[layer] = 0

    return feature_masks

In [20]:
def img_deprocess(img, img_mean=np.float32([104, 117, 123])):
    new_shape = (3, 1, 1)
    return np.dstack((img + np.reshape(img_mean, new_shape))[::-1])

In [21]:
def obj_fun(img, net, features, feature_masks, layer_weight, loss_fun, save_intermediate, save_intermediate_every, save_intermediate_path, save_intermediate_ext, save_intermediate_postprocess, loss_list=[]):
    # reshape img
    img_size = net.blobs['data'].data.shape[-3:]
    img = img.reshape(img_size)

    # save intermediate image
    t = len(loss_list)
    if save_intermediate and (t % save_intermediate_every == 0):
        img_mean = net.transformer.mean['data']
        save_path = os.path.join(save_intermediate_path, '%05d.%s' % (t, save_intermediate_ext))
        if save_intermediate_postprocess is None:
            snapshot_img = img_deprocess(img, img_mean)
        else:
            snapshot_img = save_intermediate_postprocess(img_deprocess(img, img_mean))
        PIL.Image.fromarray(snapshot_img).save(save_path)

    # layer_list
    layer_list = features.keys()
    layer_list = sort_layer_list(net, layer_list)

    # cnn forward
    net.blobs['data'].data[0] = img.copy()
    net.forward(end=layer_list[-1])

    # cnn backward
    loss = 0.
    layer_start = layer_list[-1]
    net.blobs[layer_start].diff.fill(0.)
    for j in xrange(len(layer_list)):
        layer_start_index = len(layer_list) - 1 - j
        layer_end_index = len(layer_list) - 1 - j - 1
        layer_start = layer_list[layer_start_index]
        if layer_end_index >= 0:
            layer_end = layer_list[layer_end_index]
        else:
            layer_end = 'data'
        feat_j = net.blobs[layer_start].data[0].copy()
        feat0_j = features[layer_start]
        mask_j = feature_masks[layer_start]
        layer_weight_j = layer_weight[layer_start]
        loss_j, grad_j = loss_fun(feat_j, feat0_j, mask_j)
        loss_j = layer_weight_j * loss_j
        grad_j = layer_weight_j * grad_j
        loss = loss + loss_j
        g = net.blobs[layer_start].diff[0].copy()
        g = g + grad_j
        net.blobs[layer_start].diff[0] = g.copy()
        if layer_end == 'data':
            net.backward(start=layer_start)
        else:
            net.backward(start=layer_start, end=layer_end)
        net.blobs[layer_start].diff.fill(0.)
    grad = net.blobs['data'].diff[0].copy()

    # reshape gradient
    grad = grad.flatten().astype(np.float64)
    loss_list.append(loss)
    return loss, grad

In [22]:
def normalise_img(img):
    img = img - img.min()
    if img.max() > 0:
        img = img * (255.0/img.max())
    img = np.uint8(img)
    return img

### Reconstruction Image

In [51]:
def reconstruct_image(features, net,
                      layer_weight=None, 
                      channel=None, 
                      mask=None, 
                      initial_image=None, 
                      loss_type='l2', 
                      maxiter=500, 
                      disp=True, 
                      save_intermediate=False, 
                      save_intermediate_every=1, 
                      save_intermediate_path=None,
                      save_intermediate_ext='jpg'):
    print(f'model input {model.input.shape}')
    # loss function
    loss_fun = switch_loss_fun(loss_type)
    #loss_fun = tf.keras.losses.MeanSquaredError()

    # make dir for saving intermediate
    if save_intermediate:
        if save_intermediate_path is None:
            fn_temp1 = datetime.datetime.now().strftime('%Y%m%dT%H%M%S')
            fn_name = f'recon_img_lbfgs_snapshots_{fn_temp1}'
            fp_relative = os.path.join(os.getcwd(), fn_name)
            save_intermediate_path = os.path.normpath(fp_relative)
        if not os.path.exists(save_intermediate_path):
            os.makedirs(save_intermediate_path)

    # image size
    #img_size = net.blobs['data'].data.shape[-3:]
    img_size_tuple = net.input.shape[-3:]
    print(f'image size: {img_size_tuple} {type(img_size_tuple)}')

    # num of pixel
    num_of_pix = np.prod(img_size_tuple)
    
    # image mean
    #img_mean = net.transformer.mean['data']
    img_mean_fn_str = 'ilsvrc_2012_mean.npy'
    img_mean_fp_str = os.path.normpath(os.path.join(dat_dirpath_str, 'data', img_mean_fn_str))
    if not os.path.exists(img_mean_fp_str):
        print(f'ERROR: path does not exist: {img_mean_fp_str}')
        exit()
    img_mean_ndarray = np.load(img_mean_fp_str)
    #img_mean = net.layers[0]
    print(f'image mean shape: {img_mean_ndarray.shape} of {type(img_mean_ndarray)}')
    img_mean_list = [img_mean_ndarray[0].mean(), img_mean_ndarray[1].mean(), img_mean_ndarray[2].mean()]
    img_mean_ndarray = np.float32(img_mean_list)
    
    # img bounds
    img_min_ndarray = -img_mean_ndarray
    img_max_ndarray = -img_mean_ndarray + 255.
    
    img_min0_tuple = (img_min_ndarray[0], img_max_ndarray[0])
    img_min0_list = [img_min0_tuple]
    img_min0 = np.array(img_min0_list) * num_of_pix / 3
    
    img_min1_tuple = (img_min_ndarray[1], img_max_ndarray[1])
    img_min1_list = [img_min1_tuple]
    img_min1 = np.array(img_min1_list) * num_of_pix / 3
    
    img_min2_tuple = (img_min_ndarray[2], img_max_ndarray[2])
    img_min2_list = [img_min2_tuple]
    img_min2 = np.array(img_min2_list) * num_of_pix / 3
    
    img_bounds = img_min0 + img_min1 + img_min2
    
    # initial image
    if initial_image is None:
        init_img_x = (img_size_tuple[0], img_size_tuple[1], img_size_tuple[2])
        print(f'init img x {init_img_x} {type(init_img_x)}')
        initial_image = np.random.randint(0, 256, init_img_x)
        print(f'created initial image: {initial_image.shape} {type(initial_image)}')
    if save_intermediate:
        save_fn_str = 'initial_img.png'
        save_fp_str = os.path.join(save_intermediate_path, save_fn_str)
        save_fp_str = os.path.normpath(save_fp_str)
        if not os.path.exists(save_fp_str):
            print(f'ERROR: path does not exist: {save_fp_str}')
            exit()
        init_pil_img = PIL.Image.fromarray(np.uint8(initial_image))
        init_pil_img.save(save_fp_str)

    # preprocess initial img
    #initial_image = img_preprocess(initial_image, img_mean_ndarray)
    #print('preprocessing initial image')
    img_init_ndarray = tf.keras.applications.vgg19.preprocess_input(initial_image * img_mean_ndarray)
    img_init_ndarray = img_init_ndarray.flatten()
    #print('preprocessed initial image')

    # layer_list
    layer_list = features.keys()
    
    # layer weight
    if layer_weight is None:
        weights = np.ones(len(layer_list))
        weights = np.float32(weights)
        weights = weights / weights.sum()
        layer_weight = {}
        for j, layer in enumerate(layer_list):
            layer_weight[layer] = weights[j]
    
    # feature mask
    feature_masks = create_feature_masks(features, masks=mask, channels=channel)
    
    # optimization params
    loss_list = []
    opt_params = {
        'args': (net, features, feature_masks, layer_weight, loss_fun, save_intermediate, save_intermediate_every, save_intermediate_path, save_intermediate_ext, loss_list),
        'method': 'L-BFGS-B',
        'jac': True,
        'bounds': img_bounds,
        'options': {'maxiter': maxiter, 'disp': disp},
    }
    
    # optimization
    #res = optimize.minimize(obj_fun, img_init_ndarray, **opt_params)
    #opt = tf.keras.optimizers.SGD()
    
    # recon img
    #img = res.x
    img = initial_image.reshape(img_size_tuple)
    #img = img_deprocess(img, img_mean)

    # return img
    return img, loss_list

### III) Load Input Files & Create Directories

In [39]:
img_mean = np.load(img_mean_fp)

In [40]:
model = tf.keras.applications.vgg19.VGG19(include_top=True)

In [26]:
# Feature SD estimated from true CNN features of 10000 images
feat_std_dict = scipy.io.loadmat(feat_std_fp)

In [27]:
if not os.path.isdir(bids_root_dirpath_str):
    os.makedirs(bids_root_dirpath_str)

In [28]:
if not os.path.exists(results_dirpath_str):
    os.makedirs(results_dirpath_str)

### IV) Initialize Given Variables

In [29]:
# Average image of ImageNet
img_mean_ndarray = np.float32([img_mean[0].mean(), img_mean[1].mean(), img_mean[2].mean()])
print(img_mean_ndarray)

[104.00699 116.66877 122.67892]


In [30]:
#net = caffe.Classifier(prototxt_file, model_file, mean=img_mean, channel_swap=channel_swap)
print(f'initial input shape: {model.input.shape}')
#model.layers[0].shape = (10, 3, 224, 224)
print(f'change input shape:  {model.input.shape}')

#h, w = net.blobs['data'].data.shape[-2:]
h, w = model.input.shape[1:3]
print(f'height: {h} and width: {w}')

#net.blobs['data'].reshape(1, 3, h, w)
model.layers[0].shape = (1, 3, h, w)
print(f'model reshaped: {model.input.shape}')

initial input shape: (None, 224, 224, 3)
change input shape:  (None, 224, 224, 3)
height: 224 and width: 224
model reshaped: (None, 224, 224, 3)


In [31]:
# Initial image for the optimization (here we use the mean of ilsvrc_2012_mean.npy as RGB values)
img_init_ndarray = np.zeros((h, w, 3), dtype='float32')
img_init_ndarray[:, :, 0] = img_mean_ndarray[2].copy()
img_init_ndarray[:, :, 1] = img_mean_ndarray[1].copy()
img_init_ndarray[:, :, 2] = img_mean_ndarray[0].copy()
print(f'initial image shape: {img_init_ndarray.shape}')

initial image shape: (224, 224, 3)


In [32]:
# CNN Layers (all conv and fc layers)
layers = []
count = 0
pool_count = 1
block_count = 1
for layer in model.layers:
    if layer.__class__.__name__ == 'InputLayer':
        print(f'layer {count}: {layer.name}')
    elif layer.__class__.__name__ == 'Conv2D':
        sections = layer.name.split('_')
        block = sections[0]
        blockNum = block[-1]
        if int(blockNum) > block_count:
            block_count = int(blockNum)
        layerNum = sections[1][-1]
        val = 'conv' + blockNum + '_' + layerNum
        print(f'layer {count}: {layer.name} is {val}')
        layers.append(val)
    elif layer.__class__.__name__ == 'MaxPooling2D':
        sections = layer.name.split('_')
        block = sections[0]
        blockNum = block[-1]
        if int(blockNum) > block_count:
            block_count = int(blockNum)
        val = 'pool' + blockNum
        print(f'layer {count}: {layer.name} is {val}')
        layers.append(val)
        pool_count = pool_count + 1
    elif layer.__class__.__name__ == 'Flatten':
        print(f'layer {count}: Flatten')
    elif layer.__class__.__name__ == 'Dense':
        block_count = block_count + 1
        val = 'fc' + str(block_count)
        print(f'layer {count}: {layer.name} is {val}')
        layers.append(val)
    else:
        print(f'Error finding layer {count}: {layer.name}')
    count = count + 1

layer 0: input_1
layer 1: block1_conv1 is conv1_1
layer 2: block1_conv2 is conv1_2
layer 3: block1_pool is pool1
layer 4: block2_conv1 is conv2_1
layer 5: block2_conv2 is conv2_2
layer 6: block2_pool is pool2
layer 7: block3_conv1 is conv3_1
layer 8: block3_conv2 is conv3_2
layer 9: block3_conv3 is conv3_3
layer 10: block3_conv4 is conv3_4
layer 11: block3_pool is pool3
layer 12: block4_conv1 is conv4_1
layer 13: block4_conv2 is conv4_2
layer 14: block4_conv3 is conv4_3
layer 15: block4_conv4 is conv4_4
layer 16: block4_pool is pool4
layer 17: block5_conv1 is conv5_1
layer 18: block5_conv2 is conv5_2
layer 19: block5_conv3 is conv5_3
layer 20: block5_conv4 is conv5_4
layer 21: block5_pool is pool5
layer 22: Flatten
layer 23: fc1 is fc6
layer 24: fc2 is fc7
layer 25: predictions is fc8


In [33]:
opts_dict = {
    'loss_type': 'l2', # The loss function type: {'l2','l1','inner','gram'}
    'maxiter': max_iteration, # The maximum number of iterations
    'initial_image': None, # (setting to None will use random noise as initial image)
    'disp': True # Display the information on the terminal or not
}
print(opts_dict)

{'loss_type': 'l2', 'maxiter': 200, 'initial_image': None, 'disp': True}


In [34]:
# Save the optional parameters
opts_json_str = json.dumps(opts_dict)
print(opts_json_str)
opts_fp_str = os.path.normpath(os.path.join(results_dirpath_str, 'options.json'))
options_file_handler = open(opts_fp_str, 'w')
options_file_handler.write(opts_json_str)
options_file_handler.close()

{"loss_type": "l2", "maxiter": 200, "initial_image": null, "disp": true}


### Main

In [52]:
# Reconstrucion
for subject, roi, image_label in product(subjects_list, rois_list, image_label_list):
    print(f'Subject: {subject} ROI: {roi} label: {image_label}')
    
    sub_save_dirpath_str = os.path.join(results_dirpath_str, subject, roi)
    sub_save_dirpath_str = os.path.normpath(sub_save_dirpath_str)
    #print(f'sub save dir: {sub_save_dirpath_str}')
    if not os.path.exists(sub_save_dirpath_str):
        os.makedirs(sub_save_dirpath_str)
    
    # Load the decoded CNN features
    #print('loading decoded CNN features')
    features = {}
    for layer in layers:
        # The file full name depends on the data structure for decoded CNN features
        layer_feat_fn_str = '' # PD: added code to use S1 if layer is pool
        layer_feat_fp_str = ''
        if layer.startswith('pool'):
            layer_feat_fn_str = f'{image_type}-{network_name_str}-{layer}-S1-{roi}-{image_label}.mat'
            layer_feat_fp_str = os.path.join(decoded_features_dirpath_str, image_type, network_name_str, layer, 'S1', roi, layer_feat_fn_str)
        else:
            layer_feat_fn_str = f'{image_type}-{network_name_str}-{layer}-{subject}-{roi}-{image_label}.mat'
            layer_feat_fp_str = os.path.join(decoded_features_dirpath_str, image_type, network_name_str, layer, subject, roi, layer_feat_fn_str)
        layer_feat_fp_str = os.path.normpath(layer_feat_fp_str)
        #print(f'loading layer feature file: {layer_feat_fp_str}')
        layer_feat_dict = scipy.io.loadmat(layer_feat_fp_str)
        layer_feat_ndarray = layer_feat_dict['feat']
        if 'fc' in layer:
            layer_feat_ndarray = layer_feat_ndarray.reshape(layer_feat_ndarray.size)
        
        # Correct the norm of the decoded CNN features
        feat_std_npf32 = estimate_cnn_feat_std(layer_feat_ndarray)
        layer_feat_std_ndarray = feat_std_dict[layer]
        feat_ndarray = (layer_feat_ndarray / feat_std_npf32) * layer_feat_std_ndarray
        features.update({layer: feat_ndarray})
    #print('loaded decoded CNN features')
    
    # Weight of each layer in the total loss function
    
    # Norm of the CNN features for each layer
    #print('normalizing features for layers')
    feat_list = []
    for layer in layers:
        feat_list.append(np.linalg.norm(features[layer]))
    feat_norm = np.array(feat_list, dtype='float32')
    #print('normalized features for layers')
    
    # Use the inverse of the squared norm of the CNN features as the weight for each layer
    weights = 1. / (feat_norm ** 2)
    
    # Normalise the weights such that the sum of the weights = 1
    weights = weights / weights.sum()
    layer_weight = dict(zip(layers, weights))
    
    opts_dict.update({'layer_weight': layer_weight})
    
    # Reconstruction
    snapshots_dir = os.path.join(sub_save_dirpath_str, 'snapshots', f'image-{image_label}')
    snapshots_dir = os.path.normpath(snapshots_dir)
    #print(f'snapshots dir: {snapshots_dir}')
    if not os.path.exists(snapshots_dir):
        print(f'ERROR: path does not exist: {snapshots_dir}')
        exit()
    
    recon_img_ndarray, loss_list = reconstruct_image(features, model, save_intermediate=True, save_intermediate_path=snapshots_dir, **opts_dict)
    print(f'recon image: {recon_img_ndarray.shape} {type(recon_img_ndarray)}')
    
    # Save the results
    recon_img_fn_str = f'normalized_recon_img_{image_label[3:]}.jpg'
    recon_img_fp_str = os.path.normpath(os.path.join(sub_save_dirpath_str, recon_img_fn_str))
    print(f'reconstructed image filepath: {recon_img_fp_str}')
    
    # To better display the image, clip pixels with extreme values
    # (0.02% of pixels with extreme low values and 0.02% of the pixels with extreme high values).
    # And then normalise the image by mapping the pixel value to be within [0,255].
    recon_img_ndarray = clip_extreme_value(recon_img_ndarray, pct=4)
    print(f'clip recon image: {recon_img_ndarray.shape} {type(recon_img_ndarray)}')
    
    # Normalize Reconstructed Image
    recon_img_ndarray = normalise_img(recon_img_ndarray)
    print(f'norm recon image: {recon_img_ndarray.shape} {type(recon_img_ndarray)}')
    
    # Create Image
    recon_pilimage = PIL.Image.fromarray(np.uint8(recon_img_ndarray))
    recon_pilimage.save(recon_img_fp_str)
    print('')

Subject: S1 ROI: VC label: Img0002
model input (None, 224, 224, 3)
image size: (224, 224, 3) <class 'tensorflow.python.framework.tensor_shape.TensorShape'>
image mean shape: (3, 256, 256) of <class 'numpy.ndarray'>
init img x (224, 224, 3) <class 'tuple'>
created initial image: (224, 224, 3) <class 'numpy.ndarray'>
recon image: (224, 224, 3) <class 'numpy.ndarray'>
reconstructed image filepath: C:\Users\PaulDRP\source\repos\eeg1\results\S1\VC\normalized_recon_img_0002.jpg
clip recon image: (224, 224, 3) <class 'numpy.ndarray'>
norm recon image: (224, 224, 3) <class 'numpy.ndarray'>

Subject: S1 ROI: VC label: Img0011
model input (None, 224, 224, 3)
image size: (224, 224, 3) <class 'tensorflow.python.framework.tensor_shape.TensorShape'>
image mean shape: (3, 256, 256) of <class 'numpy.ndarray'>
init img x (224, 224, 3) <class 'tuple'>
created initial image: (224, 224, 3) <class 'numpy.ndarray'>
recon image: (224, 224, 3) <class 'numpy.ndarray'>
reconstructed image filepath: C:\Users\Pau